In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

#讀取圖片，並將其樣本數轉乘10304*1
def img2vector(image):
    img=cv2.imread(image,0)#讀取圖片
    rows,cols=np.shape(img) #img
 #   imgVector = np.zeros((rows*cols,1))
    imgVector = np.reshape(img,(rows*cols,1))
    return imgVector


orlpath=r"C:\Users\kccho\Desktop\test"

#讀入人臉庫，每個人都可以隨時選擇 k(k=5) 張作為訓練集，其餘作為測試集
def load_orl(k):
    '''
    對訓練數據集初始化，用0填充進行，每張圖片都定為112*92,
    現在共有40個人，每個人都選擇k張，則整個訓練集大小為40*k,112*92
    '''
    train_face   = np.zeros( (92*112 , 40*k) )
    train_label  = np.zeros(  40*k  )          #[0,0,.....0](共40*k个0)
    
    test_face    = np.zeros( (92*112 , 40*(10-k)) )
    test_label   = np.zeros(  40*(10-k) )
    
    #sample=random.sample(range(10),k)#每个人都有的10张照片中，随机选取k张作为训练样本(10个里面随机选取K个成为一个列表)
   
    a=random.sample(range(1,11),10)
    
    for i in range(0,39):#共有40個人
        n1= i+1
        for j in range(0,9):#每个人都有10张照片
            
            
            image = orlpath + '/s' + str(n1) + '/' + str(a[j]) + '.pgm'
            
            
            #讀取圖片     
            img=img2vector(image)
            
            if j<k:              
                #训練集
                train_face[:,i*k+j] = img[:,0] #0~4 5~9 10~14... 欄
                train_label[i*k+j] =n1
            else:
                #測試集     
                test_face[:,i*(10-k)+(j-k)] = img[:,0]
                test_label[i*(10-k)+(j-k)] = n1
                
    return train_face,train_label,test_face,test_label


#Principle component analysis              
def PCA(data,r):
    data=np.float32(np.mat(data))
    rows=np.shape(data)[0]
    cols=np.shape(data)[1]
    
  # print(np.shape(data)[1])
    data_mean=np.mean(data,1)#column求平均值
    
    A5=data-np.tile(data_mean,(1,cols))#将所有样例减去对应均值得到A
 
    C=A5.T*A5#得到共變異矩陣

  #  D,V=np.linalg.eig(C)#求协方差矩阵的特征值和特征向量
    D=np.linalg.eig(C)[0]
    V=np.linalg.eig(C)[1]

    V_r=V[:,0:r]#按列取前r个特征向量

    V_r=A5*V_r#小矩阵特征向量向大矩阵特征向量过渡
    
      
 #   for i in range(r):
 #       V_r[i,:]=V_r[i,:]/np.linalg.norm(V_r[i,:])#特征向量归一化

    final_data=A5.T*V_r

    return final_data, data_mean, V_r


def print_(A,B,C,D):
    print("正確:",B)
    print("不正確:",A)
    print("混淆矩陣:\n",D)
    accuracy = float(B)/C
    print("準確度 :",accuracy*100,"%" )
    print("\n\n")
    
    
#人臉辨識
def face_recognition():
    r1=[10,20,30,40,50] #維度 10~50
    
    
    for i in range(5):   
        print("當降維到%d時"%(r1[i]))
    
        train_face, train_label, test_face, test_label =  load_orl(5)#得到訓練集(分成一半)
        
        #利用PCA算法进行训練
        data_train_new,  data_mean,  V_r = PCA(train_face,r1[i])      
        num_train       = data_train_new.shape[0]                    #训练脸总数
        num_test        = test_face.shape[1]                         #测试脸总数


        temp_face       = test_face - np.tile(data_mean,(1,num_test))
        data_test_new   = temp_face.T*V_r                              #得到测试脸在特征向量下的数据
        
        data_test_new   = np.array(data_test_new)                    # mat change to array
        data_train_new  = np.array(data_train_new)
    
        #测试准确度
        true_num = 0
        matrix=np.zeros( (41 , 41) )
        no=0
        
        #利用KNN
        for i in range(num_test):
            testFace           = data_test_new[i,:]
            
            diffMat            = data_train_new - np.tile(testFace,(num_train,1))#训练数据与测试脸之间距离
            
            sqDiffMat          = diffMat**2
            
            sqDistances        = sqDiffMat.sum(axis=1)#按欄求和
            
            sortedDistIndicies = sqDistances.argsort()#对向量从小到大排序，使用的是索引值,得到一个向量
            
            indexMin           = sortedDistIndicies[0]#距离最近的索引

            
            if train_label[indexMin] == test_label[i]: 
                a1=train_label[indexMin].astype(int)
                b1=test_label[i].astype(int)
                true_num =   true_num + 1
                matrix[a1,b1]=matrix[a1,b1]+1
            else:
                a2=train_label[indexMin].astype(int)
                b2=test_label[i].astype(int)
                no=no+1
                matrix[a2,b2]=  matrix[a2,b2]+1
                
        total=0      
        
        for i in range(0,40):
            total=total+matrix[i,i]
        print_(no,true_num,num_test,matrix)        
 
    
    
         
if __name__ =='__main__':  
    face_recognition() 

當降維到10時
正確: 176
不正確: 24
混淆矩陣:
 [[44.  0.  0. ...  0.  0.  0.]
 [ 0.  3.  0. ...  0.  0.  0.]
 [ 0.  0.  4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  3.  0.  0.]
 [ 0.  0.  0. ...  0.  3.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
準確度 : 88.0 %



當降維到20時
正確: 182
不正確: 18
混淆矩陣:
 [[44.  0.  0. ...  0.  0.  0.]
 [ 0.  3.  0. ...  0.  0.  0.]
 [ 0.  0.  4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2.  0.  0.]
 [ 0.  0.  0. ...  0.  3.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
準確度 : 91.0 %



當降維到30時
正確: 186
不正確: 14
混淆矩陣:
 [[44.  0.  0. ...  0.  0.  0.]
 [ 0.  3.  0. ...  0.  0.  0.]
 [ 0.  0.  4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  3.  0.  0.]
 [ 0.  0.  0. ...  0.  1.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
準確度 : 93.0 %



當降維到40時
正確: 189
不正確: 11
混淆矩陣:
 [[44.  0.  0. ...  0.  0.  0.]
 [ 0.  3.  0. ...  0.  0.  0.]
 [ 0.  0.  4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  4.  0.  0.]
 [ 0.  0.  0. ...  0.  3.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
準確度 : 94.5 %



當降維到50時
正確: 186
不正確: 14
混淆矩陣:
 [[44.  0.  0.